In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import libraries
from ipywidgets import interact, fixed, Layout
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import pandas as pd
from time import time
from datetime import datetime
from multiprocessing import Pool
from scipy.interpolate import interp1d
from scipy.stats import skew, kurtosis
import pickle
import os

import tensorflow as tf
from tensorflow import keras

from def_strain import f_strain
from normalize import Normalize, rescale_i
import Gen_XRD as genx

tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
# Set matplotlib style
plt.style.use('default')
plt.rc('font', size=9)          # controls default text sizes
plt.rc('axes', titlesize=9)     # fontsize of the axes title
plt.rc('axes', labelsize=9)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=9)    # fontsize of the tick labels
plt.rc('ytick', labelsize=9)    # fontsize of the tick labels
plt.rc('legend', fontsize=8)    # legend fontsize
plt.rc('figure', titlesize=10)  # fontsize of the figure title
plt.rc('xtick', direction = "in")
plt.rc('ytick', direction = "in")
plt.rc('xtick', top=True)
plt.rc('ytick', right=True)
# plt.rc('xtick.minor', visible=True)
# plt.rc('ytick.minor', visible=True)

In [ ]:
data_path = "DATA PATH HERE"
saved_model = "best.hdf5"
data_size = "long"

rescale_intensities = False
NAME = data_path.split("/")[-1]
save_path = "./checkpoints/" + NAME + "/"
print(save_path)
with open(save_path + NAME + ".class", 'rb') as f:
    normL = pickle.load(f)

cnn = keras.models.load_model(save_path+saved_model)

In [ ]:
# Load test set and evaluate model
raw_test = np.load(data_path + "/ndarray/test.npz")
test_data = raw_test["test_data"][:,:, np.newaxis, np.newaxis]
test_label = raw_test["test_label"][:,:]

test_label = normL.forward(test_label)
cnn.evaluate(test_data, test_label, batch_size= 2048)

In [ ]:
# Load test set, make predictions and compute losses (mse and spline mse) for each data set
new_step = 10
raw_test = np.load(data_path + "/ndarray/test.npz")
test_data = raw_test["test_data"][::new_step,:,np.newaxis,np.newaxis] # TAKE 1 in new_step POINTS
test_label = raw_test["test_label"][::new_step,:] # TAKE 1 in new_step POINTS // -2 
test_label = normL.forward(test_label)

# Make prediction
t0 = time()
pred_label = cnn.predict(test_data)
print("Prediction time: ",time() - t0)

# compute list of losses
norm = 1
loss_list = (((test_label - pred_label)/norm)**2).sum(axis=1)/np.shape(test_label)[1]
print("mse", loss_list.mean())
loss_list = np.sqrt(loss_list)

# compute losses for the splines and thickness only
norm = 1
sp_cut = -2
len_sp_loss = np.shape(test_label[:,:sp_cut])[1]
sp_loss_list = (((test_label[:,:sp_cut] - pred_label[:,:sp_cut])/norm)**2).sum(axis=1)/len_sp_loss
print("spline mse", sp_loss_list.mean())
sp_loss_list = np.sqrt(sp_loss_list)

# Scale-back prediction and test
pred_label = normL.backwards(pred_label)
test_label = normL.backwards(test_label)

In [ ]:
## Compute rms strain, rms intensity, max strain and minimum DW
if data_size == "long":
    x = np.linspace(8, -1, 1001)
elif data_size == "short":
    x = np.linspace(3, -1, 401)
material = "ZrO2.cif"
order = 4
dic_mater = genx.gen_mater(material, order)
d = dic_mater["d"]
wl = dic_mater["wl"]
N = 500
Nspline = 10
d_train = 1.2858375

nb_pred = np.shape(pred_label)[0]
range_nb = range(nb_pred)
rmsi_list = np.zeros(nb_pred)
emax_list = np.zeros(nb_pred)
dwmin_list = np.zeros(nb_pred)
emaxp_list = np.zeros(nb_pred)
dwminp_list = np.zeros(nb_pred)
maxpos_list = np.zeros(nb_pred) #NOT USED
maxposp_list = np.zeros(nb_pred) #NOT USED
rmss_list = np.zeros(nb_pred)
skew_list = np.zeros(nb_pred) #NOT USED
skewp_list = np.zeros(nb_pred) #NOT USED
kurt_list = np.zeros(nb_pred) #NOT USED
kurtp_list = np.zeros(nb_pred) #NOT USED

def rmsi_emax_dwmin(i):
    #load the ith test data set and the associated predictions
    loc_data = test_data[i,:,0,0]
    loc_label = np.copy(test_label[i])
    loc_labelp = np.copy(pred_label[i])
    loc_label[:Nspline] *= d / d_train
    loc_labelp[:Nspline] *= d / d_train
    
    #extract the parameters
    loc_scale_dw = np.abs(loc_label[Nspline])
    loc_sp = np.abs(loc_label[:Nspline])
    loc_t = np.abs(loc_label[-3])
    
    loc_scale_dwp = np.abs(loc_labelp[Nspline])
    loc_spp = np.abs(loc_labelp[:Nspline])
    loc_tp = np.abs(loc_labelp[-3])
    loc_fwhmp = loc_labelp[-2]
    loc_dynp = loc_labelp[-1]
    
    #compute strain max, max position [NOT USED], skewness [NOT USED] and kurtosis [NOT USED]
    loc_z = np.arange(N + 1) * loc_t/N
    depth = loc_t - loc_z
    loc_strain = f_strain(loc_z, loc_sp, loc_t, "B-splines smooth") * 100
    emax = loc_strain.max()

    maxpos = depth[loc_strain==loc_strain.max()][0] #NOT USED
    strain_skew = skew(loc_strain) #NOT USED
    strain_kurt = kurtosis(loc_strain) #NOT USED
        
    loc_zp = np.arange(N + 1) * loc_tp/N
    depthp = loc_tp - loc_zp
    loc_strainp = f_strain(loc_zp, loc_spp, loc_tp, "B-splines smooth") * 100
    emaxp = loc_strainp.max()
    maxposp = depthp[loc_strainp==loc_strainp.max()][0] #NOT USED
    strainp_skew = skew(loc_strainp) #NOT USED
    strainp_kurt = kurtosis(loc_strainp) #NOT USED
    
    #compute rms strain
    if loc_t >= loc_tp:
        f_interp = interp1d(loc_tp-loc_zp, loc_strainp, kind="cubic", fill_value=0, bounds_error=False)
        loc_z_interp = loc_z
        loc_t_interp = loc_t
        loc_strainp_interp = f_interp(loc_t_interp-loc_z_interp)
        loc_strain_interp = loc_strain
    else:
        f_interp = interp1d(loc_t - loc_z, loc_strain, kind="cubic", fill_value=0, bounds_error=False)
        loc_z_interp = loc_zp
        loc_t_interp = loc_tp
        loc_strain_interp = f_interp(loc_t_interp-loc_z_interp)
        loc_strainp_interp = loc_strainp

    norm = loc_strain_interp.max()
    rmss = ((loc_strain_interp - loc_strainp_interp)/norm)**2
    rmss = np.sqrt(rmss.mean())
   
    #compute DW min
    DWmin = np.exp(-loc_scale_dw)
    DWminp = np.exp(-loc_scale_dwp)
    
    #compute rmsi
    loc_xp, loc_datap = genx.XRDfromNN(dic_mater, order, loc_fwhmp, 0.001, 1, 10**loc_dynp, loc_labelp, data_size=data_size)

    d_data=loc_data[1:]-loc_data[:-1]
    index_lo=(np.where(d_data!=0.0))[0][0]
    index_hi=(np.where(d_data!=0.0))[0][-1]
    
    error = np.sqrt(((loc_data[index_lo:index_hi+1]-loc_datap[index_lo:index_hi+1])**2).mean())
    return error, emax, emaxp, DWmin, DWminp, maxpos, maxposp, rmss, strain_skew, strainp_skew, strain_kurt, strainp_kurt

pool = Pool()
for i, out in enumerate(pool.imap(rmsi_emax_dwmin, range_nb)):
    rmsi_list[i]=out[0]
    emax_list[i]=out[1]
    emaxp_list[i]=out[2]
    dwmin_list[i] = out[3]
    dwminp_list[i] = out[4] #NOT USED
    maxpos_list[i] = out[5] #NOT USED
    maxposp_list[i] = out[6] #NOT USED
    rmss_list[i] = out[7] 
    skew_list[i] = out[8] #NOT USED
    skewp_list[i] = out[9] #NOT USED
    kurt_list[i] = out[10] #NOT USED
    kurtp_list[i] = out[11] #NOT USED
    
pool.close()
pool.join()
    
# generate list of the predicted parameters
t_list = test_label[:,11]
tp_list = pred_label[:,11]
fwhm_list = test_label[:,-2]
fwhmp_list = pred_label[:,-2]
dyn_list = test_label[:,-1]
dynp_list = pred_label[:,-1]

In [ ]:
# Sort lists according to criterion and plot
sorted_args = np.argsort(rmss_list)

sp_loss_list = sp_loss_list[sorted_args]
loss_list = loss_list[sorted_args]
emax_list = emax_list[sorted_args]
emaxp_list = emaxp_list[sorted_args]
dwmin_list = dwmin_list[sorted_args]
dwminp_list = dwminp_list[sorted_args]
rmsi_list = rmsi_list[sorted_args]
t_list = t_list[sorted_args]
tp_list = tp_list[sorted_args]
fwhm_list = fwhm_list[sorted_args]
fwhmp_list = fwhmp_list[sorted_args]
dyn_list = dyn_list[sorted_args]
dynp_list = dynp_list[sorted_args]

rmss_list = rmss_list[sorted_args]

test_label = test_label[sorted_args]
test_data = test_data[sorted_args] 
pred_label = pred_label[sorted_args]

%matplotlib notebook
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,4))

@interact(nb = widgets.IntSlider(min=0, max=len(pred_label), step=1, value=0, layout=Layout(width='90%'), continuous_update=False))
def figs(nb):
    print("rms error =",loss_list[nb])
    print("rms intensity",rmsi_list[nb])
    print("rms strain",rmss_list[nb])
    print()
    print("emax predicted/theoretical =", emaxp_list[nb],"/",emax_list[nb])
    print("DWmin predicted/theoretical =", dwminp_list[nb],"/",dwmin_list[nb])
    print("position predicted/theoretical =", maxposp_list[nb],"/",maxpos_list[nb])
    print("thickness predicted/theoretical =",t_list[nb],"/",tp_list[nb])
    print()
    print("fwhm predicted/theoretical =",fwhmp_list[nb],"/",fwhm_list[nb])
    print("dynamic predicted/theoretical =",dynp_list[nb],"/",dyn_list[nb])


    data = test_data[nb,:,0,0]
    label = test_label[nb]
    labelp = np.copy(pred_label[nb])

#compute the intensity predicted from the predicted labels
#necessary information: material, order, width, eta
    labelp[:Nspline] *= d / d_train
    fwhmp = labelp[-2]
    dynp = labelp[-1]
# Compute and intensity   
    xp, datap = genx.XRDfromNN(dic_mater, order, fwhmp, 0.001, 1, 10**dynp, labelp, data_size=data_size)
    if rescale_intensities:
        norm_i = datap.min()
        datap -= norm_i
 
    d_data=data[1:]-data[:-1]
    xlim_lo=(np.where(d_data!=0.0))[0][0]
    xlim_hi=(np.where(d_data!=0.0))[0][-1]
    e=-x * d
    ax1.cla()
    ax1.plot(e, data, "darkgrey")
    ax1.plot(-xp* d, datap, "tab:blue")
    ax1.set_xlim(e[xlim_lo]-0, e[xlim_hi]+0)
    ax1.set_ylim(data.min(),data.max())
    ax1.set_xlabel("$\Delta Q / Q$ (%)")
    ax1.set_ylabel("Log(Intensity)")
# Compute and plot strain and DW     
    t = label[-3]
    scale_dw = label[Nspline]
    sp = label[:Nspline]
    z = np.arange(N + 1) * t/N
    strain = f_strain(z, sp, t, "B-splines smooth") * 100
    DW = np.exp(-scale_dw * ((strain / strain.max()) ** 2))

    tp = np.abs(labelp[-3])
    scale_dwp = np.abs(labelp[Nspline])
    spp = (labelp[:Nspline]) ### TEMPORARY
    zp = np.arange(N + 1) * tp/N
    strainp = f_strain(zp, spp, tp, "B-splines smooth") * 100
    DWp = np.exp(-scale_dwp * ((strainp / strainp.max()) ** 2))
#     print("Strain max =", strainp.max() )

    ax2.cla()  
    ax2.plot(t-z, strain, color="darkgrey", linestyle="-")
    ax2.plot(t-z, DW, color="darkgrey", linestyle=":")
    ax2.plot(tp-zp, strainp, color='tab:blue', linestyle="-")
    ax2.plot(tp-zp, DWp, color='tab:blue', linestyle=":")
    ax2.set_xlim(0, max(t,tp))
    ax2.set_ylim(min(0, strainp.min()), max(strain.max(),strainp.max()))
    ax2.set_xlabel("Depth (nm)")
    ax2.set_ylabel("Strain (%)")

    fig.show()

In [ ]:
index_list = [14999, 14998, 14995, 14980]

fig = plt.figure(figsize=(3.5, 3.25), constrained_layout=True)
spec = fig.add_gridspec(4, 2, width_ratios=[3,2])
ax1 = fig.add_subplot(spec[:,0])
for i, nb in enumerate(index_list[::]):
    ax = fig.add_subplot(spec[i,1])
    
    data = test_data[nb,:,0,0]
    label = test_label[nb]
    labelp = np.copy(pred_label[nb])

# compute the intensity predicted from the predicted labels
# necessary information: material, order, width, eta
    labelp[:Nspline] *= d / d_train
    fwhmp = labelp[-2]
    dynp = labelp[-1]
# Compute and plot intensity   
    xp, datap = genx.XRDfromNN(dic_mater, order, fwhmp, 0.001, 1, 10**dynp, labelp, data_size=data_size)

    d_data=data[1:]-data[:-1]
    xlim_lo=(np.where(d_data!=0.0))[0][0]
    xlim_hi=(np.where(d_data!=0.0))[0][-1]

    e=-x * d
    ax1.plot(e[xlim_lo:xlim_hi+1], data[xlim_lo:xlim_hi+1] - i*2, "silver", linewidth = 2, label=(i//1)*"_"+"Test data")
    ax1.plot(-xp[xlim_lo:xlim_hi+1]* d, datap[xlim_lo:xlim_hi+1] - i*2, "tab:blue", linewidth=0.75, label=(i//1)*"_"+"Comp. from prediction")
    
# Compute and plot strain and DW     
    t = label[-3]
    scale_dw = label[Nspline]
    sp = label[:Nspline]
    z = np.arange(N + 1) * t/N
    strain = f_strain(z, sp, t, "B-splines smooth") * 100 #* d / (1 * wl)
    DW = np.exp(-scale_dw * ((strain / strain.max()) ** 2) * np.log10(np.sqrt(t)))

    tp = np.abs(labelp[-3])
    scale_dwp = np.abs(labelp[Nspline])
    spp = (labelp[:Nspline]) ### TEMPORARY
    zp = np.arange(N + 1) * tp/N
    strainp = f_strain(zp, spp, tp, "B-splines smooth") * 100 #* d / (1 * wl)
    DWp = np.exp(-scale_dwp * ((strainp / strainp.max()) ** 2))# * np.log10(np.sqrt(tp)))

    ax.plot(t-z, strain, color="silver", linestyle="-", linewidth = 3, label=(i//1)*"_"+"Ground truth")
    ax.plot(tp-zp, strainp, color='tab:blue', linestyle="-", linewidth=0.75, label=(i//1)*"_"+"Prediction")
    
    ax.set_xlim(0, max(t,tp))
    ax.xaxis.set_major_locator(MaxNLocator(3))
    ax.yaxis.set_major_locator(MaxNLocator(3)) 
    ax.set_ylim(min(0, strainp.min()), max(strain.max(),strainp.max())+0.5)
    ax.set_ylabel("$e$ (%)")
    
    secax = ax.twinx()
    secax.plot(t-z, DW, color="silver", linestyle=(0,(1,0.5)), linewidth=2)
    secax.plot(tp-zp, DWp, color='tab:blue', linestyle=(0,(1,1)), linewidth=0.75)
    secax.set_ylim(0,1.1)
    secax.set_ylabel("$DW$")
    secax.set_yticks([0,0.5,1])

ax.set_xlabel("Depth (nm)")

ax1.set_xlim(-3,1)
ax1.set_xlabel("$(Q - Q_B) / Q_B$ (%)")
ax1.set_ylabel("Log(intensity)")
ax1.set_yticks(-np.arange(11)[::-1])
ax1.set_yticklabels([])

ax1.xaxis.set_major_locator(MaxNLocator(4))


fig.align_ylabels()

In [ ]:
#Full Plot: thickness, strain max, DWmin, z_strain, H, DR (compact, no hist, no zmax)
alpha = 0.05
t_range=(100, 1010)


fig = plt.figure(figsize=(7, 3.75), constrained_layout=True)
spec = fig.add_gridspec(3, 4)

ax1 = fig.add_subplot(spec[0,0])
ax7 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[1,0])
ax9 = fig.add_subplot(spec[1,1])
ax5 = fig.add_subplot(spec[2,0])
ax11 = fig.add_subplot(spec[2,1])
ax2 = fig.add_subplot(spec[0,2])
ax8 = fig.add_subplot(spec[0,3])
ax4 = fig.add_subplot(spec[1,2])
ax10 = fig.add_subplot(spec[1,3])
ax6 = fig.add_subplot(spec[2,2])
ax12 = fig.add_subplot(spec[2,3])

# Thickness data
ydata = tp_list
yerr = np.abs((tp_list-t_list)/t_list)
ax1.scatter(t_list, ydata, marker = '.', alpha=alpha, color="tab:blue")
ax1.plot([0,1050],[0,1050], 'silver')
ax1.set_xlabel(r"$t$ (nm)")
ax1.set_ylabel(r"$t^{(p)}$ (nm)")
ax1.set_xticks([0,500,1000])
ax1.set_yticks([0,500,1000])

hist = np.histogram(yerr, bins = 100000)
xh = hist[1][:-1]
yh = hist[0][:].cumsum()/len(yerr) 
ax2.semilogx(xh, yh, color="tab:blue")
ax2.semilogx([0.1,0.1],[-0.1,1.1], 'k:')
ax2.set_xlabel(r"$|\Delta t| / t$")
ax2.set_ylabel("Cum. fraction")
ax2.set_xlim(1e-4,2)
ax2.set_ylim(0,1)
print("Fraction of thickness with  error<0.1",yh[xh<=0.1][-1])

# Strain data
ydata = emaxp_list
yerr = np.abs((emaxp_list-emax_list)/emax_list)
ax3.scatter(emax_list, ydata, marker = '.', alpha=alpha)
ax3.set_xlabel(r"$e_{max}$ (%)")
ax3.set_ylabel(r"$e_{max}^{(p)}$ (%)")
if data_size == "long":
    ax3.plot([0,8],[0,8], 'silver')
    ax3.set_xticks([0,4,8])
    ax3.set_yticks([0,4,8])
elif data_size == "short":
    ax3.plot([0,2],[0,2], 'silver')
    ax3.set_xticks([0,1,2])
    ax3.set_yticks([0,1,2])

hist = np.histogram(yerr, bins = 100000)
xh = hist[1][:-1]
yh = hist[0][:].cumsum()/len(yerr) 
ax4.semilogx(xh, yh)
ax4.semilogx([0.1,0.1],[-0.1,1.1], 'k:')
ax4.set_xlabel(r"$|\Delta e_{max}| / e_{max}$")
ax4.set_ylabel("Cum. fraction")
ax4.set_xlim(1e-4,2)
ax4.set_ylim(0,1.)
print("Fraction of strain with  error<0.1",yh[xh<=0.1][-1])

# DW data
ydata = dwminp_list
yerr = np.abs((dwminp_list-dwmin_list)/dwmin_list)
ax5.scatter(dwmin_list, ydata, marker = '.', alpha=alpha)
ax5.plot([0,1],[0,1], 'silver')
ax5.set_xlabel(r"$DW_{min}$")
ax5.set_ylabel(r"$DW_{min}^{(p)}$")
ax5.set_xticks([0,0.5,1])
ax5.set_yticks([0,0.5,1])

hist = np.histogram(yerr, bins = 100000)
xh = hist[1][:-1]
yh = hist[0][:].cumsum()/len(yerr) 
ax6.semilogx(xh, yh)
ax6.semilogx([0.1,0.1],[-0.1,1.1], 'k:')
ax6.set_xlabel(r"$|\Delta DW_{min}| / DW_{min}$")
ax6.set_ylabel("Cum. fraction")
ax6.set_xlim(1e-4,2)
ax6.set_ylim(0,1.)
print("Fraction of DW with  error<0.1",yh[xh<=0.1][-1])

# FWHM data
ydata = fwhmp_list
yerr = np.abs((fwhmp_list-fwhm_list)/fwhm_list)
ax7.scatter(fwhm_list, ydata, marker = '.', alpha=alpha)
ax7.plot([0,0.1],[0,0.1], 'silver')
ax7.set_xlabel(r"$H$ (deg.)")
ax7.set_ylabel(r"$H^{(p)}$ (deg.)")
ax7.set_xticks([0,0.05,0.1])
ax7.set_yticks([0,0.05,0.1])

hist = np.histogram(yerr, bins = 100000)
xh = hist[1][:-1]
yh = hist[0][:].cumsum()/len(yerr) 
ax8.semilogx(xh, yh)
ax8.semilogx([0.1,0.1],[-0.1,1.1], 'k:')
ax8.set_xlabel(r"$|\Delta H |/ H$")
ax8.set_ylabel("Cum. fraction")
ax8.set_xlim(1e-4,2)
ax8.set_ylim(0,1.)
print("Fraction of fwhm with  error<0.1",yh[xh<=0.1][-1])

# Dynamic data
ydata = dynp_list
yerr = np.abs((dynp_list-dyn_list)/dyn_list)
ax9.scatter(dyn_list, ydata, marker = '.', alpha=alpha)
ax9.plot([2.9,6.1],[2.9,6.1], 'silver')
ax9.set_xlabel(r"$DR$")
ax9.set_ylabel(r"$DR^{(p)}$")
ax9.set_xticks([3,4,5,6])
ax9.set_yticks([3,4,5,6])

hist = np.histogram(yerr, bins = 100000)
xh = hist[1][:-1]
yh = hist[0][:].cumsum()/len(yerr) 
ax10.semilogx(xh, yh)
ax10.semilogx([0.1,0.1],[-0.1,1.1], 'k:')
ax10.set_xlabel(r"$|\Delta DR|/DR$")
ax10.set_ylabel("Cum. fraction")
ax10.set_xlim(1e-4,2)
ax10.set_ylim(0,1.)
print("Fraction of dyn range with  error<0.1",yh[xh<=0.1][-1])

# Rmss data
hist = np.histogram(rmss_list, bins = 100000)
xh = hist[1][:-1]
yh = hist[0][:]
yhcum = hist[0][:].cumsum()/len(rmss_list) 

ax12.semilogx(xh, yhcum, color="tab:blue")
ax12.semilogx([0.1,0.1],[-0.1,1.1], 'k:')
ax12.set_xlabel(r"$\sigma_e / e_{max}$")
ax12.set_ylabel("Cum. fraction")
ax12.set_xlim(1e-4,2)
ax12.set_ylim(0,1.)
print("Fraction of strain with  error<0.1",yhcum[xh<=0.1][-1])

# Data for boxplot
errt = ((tp_list-t_list)/t_list)
erre = ((emaxp_list-emax_list)/emax_list)
errdw = ((dwminp_list-dwmin_list)/dwmin_list)
rmss_list = rmss_list
errfwhm = ((fwhmp_list-fwhm_list)/fwhm_list)
errdr = ((dynp_list-dyn_list)/dyn_list)
error_list = [errt, erre, errdw, errfwhm, errdr]
pos=[0,1,2,3,4]
ax11.violinplot(
    error_list,
    positions=pos,
    showmeans=True,
    showextrema=False,
    points=1000,
    quantiles=[[0.1,0.90], [0.1,0.90], [0.1,0.90], [0.1,0.90], [0.1,0.90]])
ax11.set_xticks(pos)
ax11.set_xticklabels(["$t$", "$e$", "$DW$", "$H$", "$DR$"])
ax11.set_ylim(-0.3,0.3)
ax11.set_ylabel("Relative error")
    

fig.align_ylabels()
plt.show()
plt.savefig(save_path+"Pred_errors.png", dpi=300,bbox_inches="tight", pad_inches=0.2)
plt.savefig(save_path+"Pred_errors.svg", dpi=300,bbox_inches="tight", pad_inches=0.2)

In [ ]:
#rmss vs param plots
# fig, [[ax1, ax2],[ax3, ax4], [ax5, ax6]] = plt.subplots(3,2, figsize=(4,4))
# plt.tight_layout(pad=1.5)
fig = plt.figure(figsize=(3.5, 3.5), constrained_layout=True)
spec = fig.add_gridspec(3, 2)
ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[1,0])
ax3 = fig.add_subplot(spec[2,0])
# ax4 = fig.add_subplot(spec[1,1])
ax5 = fig.add_subplot(spec[0,1])
ax6 = fig.add_subplot(spec[1,1])

ax1.scatter(t_list, rmss_list, marker = '.', alpha=alpha)#, c = loss_list, cmap='hot')
ax1.set_xlabel(r"$t$ (nm)")
ax1.set_ylabel(r"$\sigma_e / e_{max}$")
ax1.set_xlim(-20,1050)
ax1.set_ylim(0,1)
ax2.scatter(emax_list,rmss_list, marker = '.', alpha=alpha)#, c = loss_list, cmap='hot')
ax2.set_xlabel(r"$e_{max}$ (%)")
ax2.set_ylabel(r"$\sigma_e / e_{max}$")
ax2.set_ylim(0,1)
# ax2.set_ylim(0,50)
ax3.scatter(dwmin_list, rmss_list, marker = '.', alpha=alpha)#, c = loss_list, cmap='hot')
ax3.set_xlabel(r"$DW_{min}$")
ax3.set_ylabel(r"$\sigma_e / e_{max}$")
ax3.set_ylim(0,1)
# ax2.set_ylim(0,50)
# ax4.scatter(maxpos_list, rmss_list, marker = '.', alpha=alpha)#, c = loss_list, cmap='hot')
# ax4.set_xlabel("Max.-strain depth (nm)")
# ax4.set_ylabel(r"$\sigma_e / e_{max}$")
# ax4.set_ylim(0,50)
ax5.scatter(fwhm_list, rmss_list, marker = '.', alpha=alpha)#, c = loss_list, cmap='hot')
ax5.set_xlabel(r"$H$ (deg.)")
ax5.set_ylabel(r"$\sigma_e / e_{max}$")
ax5.set_ylim(0,1)
ax6.scatter(dyn_list, rmss_list, marker = '.', alpha=alpha)#, c = loss_list, cmap='hot')
ax6.set_xlabel(r"$DR$")
ax6.set_ylabel(r"$\sigma_e / e_{max}$")
ax6.set_ylim(0,1)

fig.align_ylabels()
plt.show()
plt.savefig(save_path+"Rmss_vs_param.png", dpi=300,bbox_inches="tight", pad_inches=0.2)
plt.savefig(save_path+"Rmss_vs_param.svg", dpi=300,bbox_inches="tight", pad_inches=0.2)

In [ ]:
#parameter histograms
bins = 100
t_bins= 91

fig = plt.figure(figsize=(3.5, 3.5), constrained_layout=True)
spec = fig.add_gridspec(3, 2)
ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[1,0])
ax3 = fig.add_subplot(spec[2,0])
# ax4 = fig.add_subplot(spec[1,1])
ax5 = fig.add_subplot(spec[0,1])
ax6 = fig.add_subplot(spec[1,1])

hist = np.histogram(tp_list, bins = t_bins, range = t_range)
ax1.bar(hist[1][:-1], hist[0][:], width=0.02*hist[1].max(), color="tab:blue", alpha=1)
hist = np.histogram(t_list, bins = t_bins, range = t_range)
ax1.scatter(hist[1][:-1], hist[0][:], s=1, c='darkgrey')
ax1.set_xlabel(r"$t$ (nm)")
ax1.set_ylabel("$N_b$")

hist = np.histogram(emaxp_list, bins = bins)
ax2.bar(hist[1][:-1], hist[0][:], width=0.02*hist[1].max())
hist = np.histogram(emax_list, bins = bins)
ax2.scatter(hist[1][:-1], hist[0][:], s=1, c='darkgrey')
ax2.set_xlabel(r"$e_{max}$ (%)")
ax2.set_ylabel("$N_b$")

hist = np.histogram(dwminp_list, bins = bins)
ax3.bar(hist[1][:-1], hist[0][:], width=0.02*hist[1].max())
hist = np.histogram(dwmin_list, bins = bins)
ax3.scatter(hist[1][:-1], hist[0][:], s=1, c='darkgrey')
ax3.set_xlabel(r"$DW_{min}$")
ax3.set_ylabel("$N_b$")

hist = np.histogram(fwhmp_list, bins = 100)
ax5.bar(hist[1][:-1], hist[0][:], width=0.02*hist[1].max())
hist = np.histogram(fwhm_list, bins = 100)
ax5.scatter(hist[1][:-1], hist[0][:], s=1, c='darkgrey')
ax5.set_xlabel("$H$ (deg.)")
ax5.set_ylabel("$N_b$")

hist = np.histogram(dynp_list, bins = 100)
ax6.bar(hist[1][:-1], hist[0][:], width=0.02*hist[1].max())
hist = np.histogram(dyn_list, bins = 100)
ax6.scatter(hist[1][:-1], hist[0][:], s=1, c='darkgrey')
ax6.set_xlabel("$DR$")
ax6.set_ylabel("$N_b$")

fig.align_ylabels()
plt.show()
plt.savefig(save_path+"Param_hist.png", dpi=300,bbox_inches="tight", pad_inches=0.2)
plt.savefig(save_path+"Param_hist.svg", dpi=300,bbox_inches="tight", pad_inches=0.2)

In [ ]:
#Plot histograms for lowest and highest rmss
fig, [[ax5, ax6],[ax1, ax2],[ax3, ax4],[ax7, ax8],[ax9,ax10]] = plt.subplots(5,2, figsize=(3.5,5))
plt.tight_layout(pad=1.5)

bins = 100
t_bins = 96
fraction = 1000
bar_width = 0.02

ax5.set_title("$10^3$ lowest $\sigma_e / e_{max}$")
ax6.set_title("$10^3$ highest $\sigma_e / e_{max}$")

my_list = emax_list
hist = np.histogram(my_list[:fraction], bins = bins)
ax1.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax1.set_xlabel(r"$e_{max}$ (%)")
ax1.set_ylabel(r"$N_b$")

hist = np.histogram(my_list[-fraction:], bins = bins)
ax2.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax2.set_xlabel(r"$e_{max}$ (%)")

if data_size == "long":
    ax1.set_xticks([0,2.5,5,7.5])
    ax2.set_xticks([0,2.5,5,7.5])
elif data_size == "short":
    ax1.set_xticks([0,1,2])
    ax2.set_xticks([0,1,2])
    
my_list = dwmin_list
hist = np.histogram(my_list[:fraction], bins = bins)
ax3.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax3.set_xlabel(r"$DW_{min}$")
ax3.set_ylabel(r"$N_b$")
ax3.set_xticks([0,0.5,1])

hist = np.histogram(my_list[-fraction:], bins = bins)
ax4.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax4.set_xlabel(r"$DW_{min}$")
ax4.set_xticks([0,0.5,1])
# ax4.set_ylabel("$N_b$")

my_list = t_list
hist = np.histogram(my_list[:fraction], bins = t_bins)
ax5.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax5.set_xlabel(r"$t$ (nm)")
ax5.set_ylabel(r"$N_b$")
ax5.set_xticks([0,500,1000])


hist = np.histogram(my_list[-fraction:], bins = t_bins)
ax6.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax6.set_xlabel(r"$t$ (nm)")
ax6.set_xticks([0,500,1000])
# ax6.set_ylabel("$N_b$")

my_list = fwhm_list
hist = np.histogram(my_list[:fraction], bins = bins)
ax7.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax7.set_xlabel(r"$H$ (deg.)")
ax7.set_ylabel(r"$N_b$")
ax7.set_xticks([0,0.05,0.1])

hist = np.histogram(my_list[-fraction:], bins = bins)
ax8.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max())
ax8.set_xlabel(r"$H$ (deg.)")
ax8.set_xticks([0,0.05,0.1])
# ax8.set_ylabel("$N_b$")

my_list = dyn_list
hist = np.histogram(my_list[:fraction], bins = bins)
ax9.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max()/2)
ax9.set_xlabel("$DR$")
ax9.set_ylabel(r"$N_b$")
ax9.set_xticks([3,4,5,6])

hist = np.histogram(my_list[-fraction:], bins = bins)
ax10.bar(hist[1][1:], hist[0][:], width=bar_width*hist[1].max()/2)
ax10.set_xlabel("$DR$")
ax10.set_xticks([3,4,5,6])
# ax10.set_ylabel(r"$N_b$")

fig.align_ylabels()
plt.show()
plt.savefig(save_path+"Param_hist_hi_lo.png", dpi=300,bbox_inches="tight", pad_inches=0.2)
plt.savefig(save_path+"Param_hist_hi_lo.svg", dpi=300,bbox_inches="tight", pad_inches=0.2)